# e) Power and Energy Model

Calculates power and energy over the historic daily flow series.

## Procedure

For each day in the series:
- Apply the selected Environmental Assurance Categories for each channel
- Determine the EWR flows for each channel
- Calculate the canal flow
- Calculate the headpond and tailrace levels
- Calculate headlosses in the left channel and canal
- Aportion canal flows amongst the turbines
- Calculate headlosses in the turbines and net head
- Calculate turbine and generator efficiency
- Calculate power and energy

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| ewr_section_categories.csv |                                               |  EWR Assurance Category Sets            |
| ngonye_flow_daily_ewrx.csv | Notebook: d_ewr_setup             |  Daily flow data with the floww exceddance values (each 10 days) |
| headloss_leftchannel.csv   | Mott MacDonald - Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D | Headloss in the left channel by flow  |
| headloss_canal.csv         | Mott MacDonald - Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D | Headloss in the canal by flow |









## Libraries and Setup

In [6]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [7]:
input_data='./input_data/'
output_data='./output_data/'

In [8]:
model_name='Base Case'
dryrun=False #Dont write output files

In [9]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100
fixed_tailwater_level=model['FixedTailwater']#False #970

model


OutputPrefix                  base
EWRCatSet         Recommendation 1
HeadpondLift                     0
Capacity                       180
FixedTailwater                   0
Mean                        832123
P95                         717411
P90                         754939
P75                         791547
P50                         838318
P25                         875947
P10                         921510
P5                          957737
StDev                      82094.8
CoefVar                  0.0986571
aa                             NaN
Name: Base Case, dtype: object

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [10]:
daily=pd.read_csv(output_data + "ngonye_flow_daily_ewrx.csv")
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily=daily.astype({'WaterWeek': 'int32'})
daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33968 entries, 1924-10-01 to 2017-09-30
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LaggedDate        33968 non-null  object 
 1   VicFalls          33968 non-null  float64
 2   Conversion        33968 non-null  float64
 3   Flow              33968 non-null  float64
 4   Exceedance        33968 non-null  float64
 5   Year              33968 non-null  int64  
 6   Month             33968 non-null  int64  
 7   Day               33968 non-null  int64  
 8   MonthId           33968 non-null  float64
 9   WaterYear         33968 non-null  int64  
 10  WaterMonth        33968 non-null  int64  
 11  WaterDay          33968 non-null  int64  
 12  WaterWeek         33968 non-null  int32  
 13  Volume            33968 non-null  float64
 14  EWRRefExceedance  33968 non-null  float64
dtypes: float64(7), int32(1), int64(6), object(1)
memory usage: 4.0+ MB


## EWR Assurance Category Sets
Load the available EWR assurance category sets

In [11]:
category_sets=pd.read_csv(input_data + "ewr_section_categories.csv")
category_sets

,Section,Recommendation 1,All Cs,All Bs,All As
0,A,B,C,B,A
1,C,B,C,B,A
2,D,B,C,B,A
3,E,B,C,B,A
4,FG,C,C,B,A


## EWR Assurance Category Set
Select the category set given as a variable in  the setup above

In [12]:
category_set=category_sets.loc[:,['Section',category_set_name]]
category_set=category_set.rename(columns={category_set_name:'Category'})
category_set

,Section,Category
0,A,B
1,C,B
2,D,B
3,E,B
4,FG,C


Load the EWR Flow Sets which specify the required EWRs by assurance category, channel section and calendar month

In [13]:
ewr_flow_sets=pd.read_csv(input_data + "ewr_flow_sets.csv")
ewr_flow_sets=ewr_flow_sets.set_index(['Section','Category'])
ewr_flow_sets

Month  Wet  Mod_Wet  Normal  Mod_Dry  Dry
Section Category                                           
A       A            10   75       70      60       47   38
        A            11   73       69      58       45   37
        A            12   77       72      62       48   40
        A             1   83       77      66       52   46
        A             2   90       81      69       56   51
...                 ...  ...      ...     ...      ...  ...
FG      D             5   76       76      66       49   42
        D             6   70       57      46       39   31
        D             7   64       53      41       31   27
        D             8   51       43      32       25   22
        D             9   36       31      23       17   15

[240 rows x 6 columns]

## EWR Flow Sets
Select the EWR Flow Set for each channel section according to the Category Set chosen above (eg *Recommendation 1*)

In [14]:
ewrs=category_set.join(ewr_flow_sets,on=['Section','Category']).set_index(['Section','Month'])
ewrs_A=ewrs.loc['A']
ewrs_C=ewrs.loc['C']
ewrs_D=ewrs.loc['D']
ewrs_E=ewrs.loc['E']
ewrs_FG=ewrs.loc['FG']

ewrs_A


,Category,Wet,Mod_Wet,Normal,Mod_Dry,Dry
Month,,,,,,
10,B,44,41,37,31,24
11,B,42,40,36,29,23
12,B,44,42,38,31,26
1,B,49,46,41,34,29
2,B,55,49,43,36,31
3,B,68,59,49,41,36
4,B,84,70,54,45,40
5,B,71,65,55,47,42
6,B,67,62,53,45,39


## EWR Flow Bands
Annotate the daily flow series with the EWR Flow Band (Wet, Very Wet, Dry etc) - a lookup against the EWR Exceedance

In [15]:
def flowBandNo(exceed):
    if exceed <= 0.1:
        return 1
    elif  exceed <= 0.4:
        return 2
    elif  exceed <= 0.7:
        return 3
    elif  exceed <= 0.9:
        return 4
    else:
        return 5
    
def flowBandLabel(bandNo):
    if bandNo == 1:
        return 'Wet'
    elif  bandNo == 2:
        return 'Mod_Wet'
    elif  bandNo == 3:
        return 'Normal'
    elif  bandNo ==4:
        return 'Mod_Dry'
    else:
        return 'Dry'    
    
def ewrLookup(section, band, month):
    return ewrs.loc[(section,month)][band]

In [16]:

daily['EWRBandNo']=daily.apply(lambda x: flowBandNo(x['EWRRefExceedance']),axis=1) 
daily['EWRBandLabel']=daily.apply(lambda x: flowBandLabel(x['EWRBandNo']),axis=1) 

## EWR Flows

In [17]:

daily['EWRChannelA']=ewrs_A.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelC']=ewrs_C.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelD']=ewrs_D.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelE']=ewrs_E.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelFG']=ewrs_FG.lookup(daily['Month'],daily['EWRBandLabel'])


In [18]:
daily['EWRTotal']=daily['EWRChannelA']+daily['EWRChannelC']+daily['EWRChannelD']+daily['EWRChannelE']+daily['EWRChannelFG']
daily['EWRProportion']=daily['EWRTotal']/daily['Flow']

## Spill and channel flows

In [19]:
daily['FlowCanal']=np.where(daily['Flow']-daily['EWRTotal']>canal_capacity,canal_capacity,daily['Flow']-daily['EWRTotal'])
daily['FlowSpill']=np.where(daily['Flow']-daily['EWRTotal']<=canal_capacity,0,daily['Flow']-daily['EWRTotal']-canal_capacity)

daily['SpillChannelA']=daily['FlowSpill']*(daily['EWRChannelA']/daily['EWRTotal'])
daily['SpillChannelC']=daily['FlowSpill']*(daily['EWRChannelC']/daily['EWRTotal'])
daily['SpillChannelD']=daily['FlowSpill']*(daily['EWRChannelD']/daily['EWRTotal'])
daily['SpillChannelE']=daily['FlowSpill']*(daily['EWRChannelE']/daily['EWRTotal'])
daily['SpillChannelFG']=daily['FlowSpill']*(daily['EWRChannelFG']/daily['EWRTotal'])

daily['FlowChannelA']=daily['SpillChannelA']+daily['EWRChannelA']
daily['FlowChannelC']=daily['SpillChannelC']+daily['EWRChannelC']
daily['FlowChannelD']=daily['SpillChannelD']+daily['EWRChannelD']
daily['FlowChannelE']=daily['SpillChannelE']+daily['EWRChannelE']

daily['TmpA']=np.where(daily['FlowChannelA']>100,daily['FlowChannelA']-100,0)
daily['SpillChannelA']=daily['SpillChannelA']-daily['TmpA']
daily['FlowChannelA']=daily['FlowChannelA']-daily['TmpA']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpA']

daily['TmpC']=np.where(daily['FlowChannelC']>18,daily['FlowChannelC']-18,0)
daily['SpillChannelC']=daily['SpillChannelC']-daily['TmpC']
daily['FlowChannelC']=daily['FlowChannelC']-daily['TmpC']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpC']

daily['TmpD']=np.where(daily['FlowChannelD']>25,daily['FlowChannelD']-25,0)
daily['SpillChannelD']=daily['SpillChannelD']-daily['TmpD']
daily['FlowChannelD']=daily['FlowChannelD']-daily['TmpD']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpD']

daily['TmpE']=np.where(daily['FlowChannelE']>80,daily['FlowChannelE']-80,0)
daily['SpillChannelE']=daily['SpillChannelE']-daily['TmpE']
daily['FlowChannelE']=daily['FlowChannelE']-daily['TmpE']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpE']

daily['FlowChannelFG']=daily['SpillChannelFG']+daily['EWRChannelFG']
daily=daily.drop(['TmpA','TmpC','TmpD','TmpE'],axis=1)

daily['FlowLeftChannel']=daily['FlowChannelA']+daily['FlowChannelC']+daily['FlowChannelD']+daily['FlowCanal']


In [20]:

daily['Turbines']=daily.apply(lambda x: math.ceil(x['FlowCanal'] / (canal_capacity/4)),axis=1)
daily.loc[daily['FlowCanal']<=plant_capacity/180*50,'Turbines']=0

In [21]:
daily['FlowTurbine1']=np.where(daily['Turbines']>=1,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine2']=np.where(daily['Turbines']>=2,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine3']=np.where(daily['Turbines']>=3,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine4']=np.where(daily['Turbines']==4,daily['FlowCanal']/daily['Turbines'],np.nan)

## Levels, headloss and head
\begin{equation*}
Headloss Powerhouse=0.00000702Q^2 + 0.00002226Q - 0.00207422
\end{equation*}

\begin{equation*}
Tailwater Level=6.0977*log(Q)^2-27.598*log(Q)+995.82
\end{equation*}

\begin{equation}
  Headpond Level=\begin{cases}
    990.0, & \text{if $Q<5000$}.\\
    0.000081307*Q+989.6, & \text{if $Q<7500$}.\\
    0.000240*Q+988.4, & \text{otherwise}.
  \end{cases}
\end{equation}


In [22]:
def headpondLevel(flow):
    if flow<5000:
        return 990.0 + headpond_lift
    elif flow<7500:
        return 0.000081307*flow+989.6 + headpond_lift
    else:
        return 0.000240*flow+988.4 + headpond_lift

In [23]:
a=False
if a>1:
    display('ss')

In [24]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        log_flow=math.log10(flow)
        if log_flow>=3.8:
            return -21.429*log_flow**2+182.03*log_flow+597.15   
        elif log_flow>=2.3:
            return 6.5299*log_flow**2-30.155*log_flow+999.52   
        else:
            return -2.6828*log_flow**2+11.265*log_flow+952.68

In [25]:
daily['LevelTailwater']=daily['Flow'].apply(tailwaterLevel)
daily['LevelHeadpond']=daily['Flow'].apply(headpondLevel)

In [26]:
headloss_leftchannel=pd.read_csv(input_data + "headloss_leftchannel.csv")
fHeadlossLeftChannel = interpolate.interp1d(headloss_leftchannel['FlowLeftChannel'], headloss_leftchannel['HeadlossLeftChannel'])
daily['HeadlossLeftChannel']=daily['FlowLeftChannel'].apply(fHeadlossLeftChannel)


In [27]:
headloss_canal=pd.read_csv(input_data + "headloss_canal.csv")
fHeadlossCanal = interpolate.interp1d(headloss_canal['FlowCanal'], headloss_canal['HeadlossCanal'])

daily['HeadlossCanal']=daily['FlowCanal'].apply(fHeadlossCanal)
daily['LevelForebay']=daily['LevelHeadpond']-daily['HeadlossLeftChannel']-daily['HeadlossCanal']

In [28]:
def turbineHeadloss(flow):
    if np.isnan(flow):
        return np.nan
    elif flow==0:
        return 0
    else:
        return 0.00000702*flow**2+0.00002226*flow-0.00207422

In [29]:
daily['HeadlossTurbine1']=daily['FlowTurbine1'].apply(turbineHeadloss)
daily['HeadlossTurbine2']=daily['FlowTurbine2'].apply(turbineHeadloss)
daily['HeadlossTurbine3']=daily['FlowTurbine3'].apply(turbineHeadloss)
daily['HeadlossTurbine4']=daily['FlowTurbine4'].apply(turbineHeadloss)

In [30]:
daily['HeadTurbine1']=np.where(daily['FlowTurbine1']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine1'],np.nan)
daily['HeadTurbine2']=np.where(daily['FlowTurbine2']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine2'],np.nan)
daily['HeadTurbine3']=np.where(daily['FlowTurbine3']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine3'],np.nan)
daily['HeadTurbine4']=np.where(daily['FlowTurbine4']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine4'],np.nan)


## Turbine and generator efficiency

In [31]:
def turbineEfficiency(q_actual,h):
    
    #Scale flow to installed capacity
    q=q_actual*(180/plant_capacity)
    
    if np.isnan(h) or np.isnan(q):
        return np.nan
    
    if h<=10:
        return 0
    
    if q==0:
        return 0
    elif q<53.5:
        return 0.77
    elif q<55:
        return 0.78
    elif q<58.5:
        return 0.79
    elif q<62:
        return 0.80
    elif q<64:
        return 0.81
    elif q<66.5:
        return 0.82
    elif q<70:
        return 0.83
    elif q<72:
        return 0.84
    elif q<74.5:
        return 0.85
    elif q<82.5:
        return 0.86
    elif q<91.5:
        return 0.87
    elif q<100.5:
        return 0.88
    elif q<109.72:
        return 0.89
    elif q<121.77:
        return 0.90
    elif q<146.8:
        return 0.91
    elif q<169.545:
        return 0.92
    elif q<173.1:
        if h<24.65:
            return 0.92
        else:
            return 0.93
    elif q<199.8:
        return 0.93
    elif q<211.2:
        if h<23:
            return 0.93
        else:
            return 0.94
    elif q<247.05:
        return 0.94
    elif q<250:
        if h<24.16:
            return 0.94
        else:
            return 0.95
    elif q<260:
        if h<23.6:
            return 0.94
        else:
            return 0.95
    elif q<275:
        if h<22.5:
            return 0.94
        else:
            return 0.95
    else:
        return -0.0015*h**2 + 0.0565*h + 0.4

In [ ]:
daily['EffTurbine1']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine1'],x['HeadTurbine1']),axis=1)
daily['EffTurbine2']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine2'],x['HeadTurbine2']),axis=1)
daily['EffTurbine3']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine3'],x['HeadTurbine3']),axis=1)
daily['EffTurbine4']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine4'],x['HeadTurbine4']),axis=1)

In [ ]:
daily['LoadFactorTurbine1']=daily['FlowTurbine1']/250*(plant_capacity/180)
daily['LoadFactorTurbine2']=daily['FlowTurbine2']/250*(plant_capacity/180)
daily['LoadFactorTurbine3']=daily['FlowTurbine3']/250*(plant_capacity/180)
daily['LoadFactorTurbine4']=daily['FlowTurbine4']/250*(plant_capacity/180)

In [ ]:
def generatorEff(load_fact):
    if np.isnan(load_fact):
        return np.nan
    
    if load_fact==0:
        return 0
    elif load_fact<=0.45:
        return 0.96
    elif load_fact<=0.85:
        return 0.97
    else:
        return 0.98

In [ ]:
daily['EffGen1']=daily['LoadFactorTurbine1'].apply(generatorEff)
daily['EffGen2']=daily['LoadFactorTurbine2'].apply(generatorEff)
daily['EffGen3']=daily['LoadFactorTurbine3'].apply(generatorEff)
daily['EffGen4']=daily['LoadFactorTurbine4'].apply(generatorEff)

## Power and energy

In [ ]:
daily['PowerTurbine1']=daily['FlowTurbine1']*daily['HeadTurbine1']*daily['EffTurbine1']*daily['EffGen1']*9.81/1000
daily['PowerTurbine2']=daily['FlowTurbine2']*daily['HeadTurbine2']*daily['EffTurbine2']*daily['EffGen2']*9.81/1000
daily['PowerTurbine3']=daily['FlowTurbine3']*daily['HeadTurbine3']*daily['EffTurbine3']*daily['EffGen3']*9.81/1000
daily['PowerTurbine4']=daily['FlowTurbine4']*daily['HeadTurbine4']*daily['EffTurbine4']*daily['EffGen4']*9.81/1000
daily['Power']=np.nan_to_num(daily['PowerTurbine1'])+np.nan_to_num(daily['PowerTurbine2'])+np.nan_to_num(daily['PowerTurbine3'])+np.nan_to_num(daily['PowerTurbine4'])
daily['Energy']=daily['Power']*24
daily['PerfMwPerCumec']=daily['Power']/daily['Flow']

## Summarise
Aggregate into months, years, calendar months etc

In [ ]:

monthly=daily.groupby("MonthId").mean().drop(['VicFalls','Conversion','Day','WaterDay','WaterWeek','Volume','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(180*24*monthly['Days'])

monthly.tail(12)

In [ ]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(180*24*365)


In [ ]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


In [ ]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy']/1000,(1-edc.index)*100)
edc

In [ ]:
monthly_day=daily.loc[daily['Day']==15]

In [ ]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

In [ ]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','Year','MonthId','WaterMonth','VicFalls','Conversion','Day','WaterDay','Volume','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)
weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(180*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

In [ ]:
weekly.head(4)

In [ ]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

In [ ]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly

In [ ]:
#fdc=pd.DataFrame(index=np.arange(0,1.01,0.01))
#fdc=pd.merge_asof(fdc,daily.sort_values('Exceedance'),left_index=True,right_on='Exceedance').reset_index().set_index('Exceedance').drop(['Date','LaggedDate','VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)

#fdc=daily.reset_index().groupby(lambda x: np.round(x['Exceedance'],2)).mean()
daily['ExceedanceRnd']=np.round(daily['Exceedance'],2)
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

In [ ]:
if dryrun==False:
    daily.to_csv(output_data+out_prefix+'_pe_daily.csv')
    monthly.to_csv(output_data+out_prefix+'_pe_monthly.csv')
    yearly.to_csv(output_data+out_prefix+'_pe_yearly.csv')
    edc.to_csv(output_data+out_prefix+'_pe_percentiles.csv')
    monthly_day.to_csv(output_data+out_prefix+'_pe_monthly_day.csv')
    daily_slim.to_csv(output_data+out_prefix+'_pe_daily_slim.csv')
    weekly.to_csv(output_data+out_prefix+'_pe_weekly.csv')
    weekly_slim.to_csv(output_data+out_prefix+'_pe_weekly_slim.csv')
    calmonthly.to_csv(output_data+out_prefix+'_pe_calmonthly.csv')
    fdc.to_csv(output_data+out_prefix+'_pe_fdc.csv')

In [ ]:
models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

if dryrun==False:
    models.to_csv(input_data + "models.csv")

models

